# Get Soccer Players Statistics

In [ ]:
# API KEY = 4ac882eae3a8b538544ad3e20d102456

In [22]:
import requests
import pandas as pd
from typing import Dict, Any, List, Optional
import os
from dotenv import load_dotenv
from typing import Set, Union


class FootballPlayerStats:
    """
    Classe para obter estatísticas de jogadores de futebol da API-Football
    """
    
    def __init__(self, api_key: Optional[str] = None):
        """
        Inicializa o cliente da API-Football
        
        Args:
            api_key: Chave de API para API-Football. Se não fornecida, tentará carregar de variáveis de ambiente.
        """
        load_dotenv()  # Carrega variáveis de ambiente do arquivo .env
        
        self.api_key = api_key or os.getenv("API_FOOTBALL_KEY")
        if not self.api_key:
            raise ValueError("API key não fornecida. Defina API_FOOTBALL_KEY no ambiente ou passe como parâmetro.")
            
        self.base_url = "https://v3.football.api-sports.io"
        self.headers = {
            "x-apisports-key": self.api_key,
        }
    
    # Cache para armazenar dados e evitar chamadas repetidas à API
        self.cache = {
            "league_info": None,
            "teams": {},
            "players": {}
        }
        
        # ID do Campeonato Brasileiro
        self.brazilian_league_id = 71
        self.current_season = self._get_current_season()
    
    def _get_current_season(self) -> int:
        """
        Obtém a temporada atual do Campeonato Brasileiro
        
        Returns:
            Ano da temporada atual
        """
        # Podemos obter a temporada atual da API ou usar o ano atual
        # Para simplificar, vamos usar 2023 como exemplo
        return 2023
    
    def _make_api_request(self, endpoint: str, params: Dict[str, Any]) -> Dict[str, Any]:
        url = f"{self.base_url}/{endpoint}"
    
        try:
            resp = requests.get(url, headers=self.headers, params=params, timeout=30)
    
            # Rate limit
            if resp.status_code == 429:
                retry_after = int(resp.headers.get("Retry-After", "60"))
                print(f"[429] Rate limit. Esperando {retry_after}s. endpoint={endpoint} params={params}")
                time.sleep(retry_after)
                return self._make_api_request(endpoint, params)
    
            # Se der erro, printa o body (isso é o que vai te dizer o motivo do 0)
            if resp.status_code >= 400:
                print(f"[HTTP {resp.status_code}] endpoint={endpoint} params={params}")
                print("Body:", resp.text[:1000])  # corta pra não explodir o terminal
                resp.raise_for_status()
    
            data = resp.json()
    
            # Log útil quando vier sem jogadores
            if not data.get("response"):
                print(f"[EMPTY] endpoint={endpoint} params={params} results={data.get('results')}")
                if data.get("errors"):
                    print("errors:", data["errors"])
                if data.get("paging"):
                    print("paging:", data["paging"])
    
            return data
    
        except requests.RequestException as e:
            print(f"[EXCEPTION] endpoint={endpoint} params={params} err={e}")
            return {"response": [], "errors": {"exception": str(e)}}
    
    def get_brazilian_league_info(self) -> Dict[str, Any]:
        """
        Obtém informações sobre o Campeonato Brasileiro
        
        Returns:
            Informações sobre o Campeonato Brasileiro
        """
        if self.cache["league_info"]:
            return self.cache["league_info"]
        
        params = {
            "id": self.brazilian_league_id,
            "season": self.current_season
        }
        
        data = self._make_api_request("leagues", params)
        
        if data and "response" in data and data["response"]:
            self.cache["league_info"] = data["response"][0]
            return self.cache["league_info"]
        
        return {}
    
    def get_teams_in_league(self) -> List[Dict[str, Any]]:
        """
        Obtém todos os times do Campeonato Brasileiro na temporada atual
        
        Returns:
            Lista de times
        """
        params = {
            "league": self.brazilian_league_id,
            "season": self.current_season
        }
        
        data = self._make_api_request("teams", params)
        
        teams = []
        if data and "response" in data:
            teams = data["response"]
            
            # Armazena os times no cache
            for team in teams:
                team_id = team["team"]["id"]
                self.cache["teams"][team_id] = team
        
        return teams
    
    def get_players_by_team(self, team_id: int) -> List[Dict[str, Any]]:
        """
        Obtém todos os jogadores de um time
        
        Args:
            team_id: ID do time
            
        Returns:
            Lista de jogadores
        """
        if team_id in self.cache["players"]:
            return self.cache["players"][team_id]
        
        params = {
            "team": team_id,
            "season": self.current_season
        }
        
        data = self._make_api_request("players", params)
        
        players = []
        if data and "response" in data:
            players = data["response"]
            self.cache["players"][team_id] = players
        
        return players

    def _flatten_keys(self, obj: Any, prefix: str = "") -> Set[str]:
        """
        Retorna todas as chaves (incluindo aninhadas) em dot-notation.
        Ex:
          {"team": {"id": 1}} -> {"team", "team.id"}
          {"stats": [{"games": {"position": "GK"}}]} -> {"stats", "stats[]", "stats[].games", "stats[].games.position"}
        """
        keys: Set[str] = set()
    
        if isinstance(obj, dict):
            for k, v in obj.items():
                path = f"{prefix}.{k}" if prefix else str(k)
                keys.add(path)
                keys |= self._flatten_keys(v, path)
    
        elif isinstance(obj, list):
            # marca o nível de lista
            list_prefix = f"{prefix}[]" if prefix else "[]"
            keys.add(list_prefix)
    
            # tenta inferir chaves olhando alguns elementos
            for item in obj[:3]:  # limita pra não ficar caro
                keys |= self._flatten_keys(item, list_prefix)
    
        return keys


    def debug_print_endpoint_keys(self, sample_size: int = 3) -> None:
        """
        Para cada endpoint usado, imprime as chaves disponíveis nos elementos retornados.
        """
        endpoints = [
            ("leagues", {"id": self.brazilian_league_id, "season": self.current_season}),
            ("teams", {"league": self.brazilian_league_id, "season": self.current_season}),
            # players é pesado; pega só alguns times pra mapear estrutura
            ("players", None),  # params serão montados abaixo
        ]
    
        for endpoint, params in endpoints:
            print("\n" + "=" * 80)
            print(f"ENDPOINT: {endpoint}")
    
            if endpoint != "players":
                data = self._make_api_request(endpoint, params)
                resp = data.get("response", [])
                print(f"Itens retornados: {len(resp)} (mostrando até {sample_size} para inspeção)")
    
                keys: Set[str] = set()
                for item in resp[:sample_size]:
                    keys |= self._flatten_keys(item)
    
                for k in sorted(keys):
                    print(k)
    
            else:
                # Players: escolhe alguns times e imprime o mapa por time
                teams = self.get_teams_in_league()
                if not teams:
                    print("Não consegui obter times para inspecionar players.")
                    continue
    
                team_samples = teams[:min(sample_size, len(teams))]
                for t in team_samples:
                    team_id = t["team"]["id"]
                    team_name = t["team"]["name"]
    
                    print("\n" + "-" * 80)
                    print(f"PLAYERS for team: {team_name} (id={team_id})")
    
                    data = self._make_api_request("players", {"team": team_id, "season": self.current_season})
                    resp = data.get("response", [])
                    print(f"Jogadores retornados: {len(resp)} (mostrando até {sample_size} jogadores)")
    
                    keys: Set[str] = set()
                    for item in resp[:sample_size]:
                        keys |= self._flatten_keys(item)
    
                    for k in sorted(keys):
                        print(k)
    
    def build_brazilian_league_players_dict(self) -> Dict[str, Dict[str, List[Dict[str, str]]]]:
        """
        Constrói um dicionário com todos os times e jogadores do Campeonato Brasileiro
        
        Returns:
            Dicionário com times e jogadores
        """
        # Obtém informações da liga
        league_info = self.get_brazilian_league_info()
        if not league_info:
            print("Não foi possível obter informações sobre o Campeonato Brasileiro")
            return {}
        
        print(f"Obtendo dados do {league_info['league']['name']} - Temporada {self.current_season}")
        
        # Obtém todos os times
        teams = self.get_teams_in_league()
        if not teams:
            print("Não foi possível obter os times do Campeonato Brasileiro")
            return {}
        
        print(f"Encontrados {len(teams)} times no campeonato")
        
        # Dicionário para armazenar os resultados
        brazilian_league_dict = {
            "league_name": league_info['league']['name'],
            "season": self.current_season,
            "teams": {}
        }
        
        # Para cada time, obtém os jogadores
        for team in teams:
            team_id = team["team"]["id"]
            team_name = team["team"]["name"]
            
            print(f"Obtendo jogadores do {team_name}...")
            print(team.keys())
            
            players = self.get_players_by_team(team_id)
            
            # Cria lista de jogadores com nome e posição
            players_list = []
            for player in players:
                player_info = {
                    "id": player["player"]["id"],
                    "name": player["player"]["name"],
                    "position": player["statistics"][0]["games"]["position"] if player["statistics"] else "Desconhecido",
                    "age": player["player"]["age"],
                    "nationality": player["player"]["nationality"]
                }
                players_list.append(player_info)
            
            # Adiciona o time e seus jogadores ao dicionário
            brazilian_league_dict["teams"][team_name] = {
                "team_id": team_id,
                "players": players_list
            }
            
            # Pausa para evitar rate limit
            time.sleep(7)
        
        return brazilian_league_dict

In [5]:
import time

In [24]:
client = FootballPlayerStats()

data = client.build_brazilian_league_players_dict()

print(data["league_name"])
print(data["season"])

for team, info in data["teams"].items():
    print(team, len(info["players"]))

Obtendo dados do Serie A - Temporada 2023
Encontrados 20 times no campeonato
Obtendo jogadores do Bahia...
dict_keys(['team', 'venue'])
Obtendo jogadores do Internacional...
dict_keys(['team', 'venue'])
Obtendo jogadores do Botafogo...
dict_keys(['team', 'venue'])
Obtendo jogadores do Palmeiras...
dict_keys(['team', 'venue'])
Obtendo jogadores do Fluminense...
dict_keys(['team', 'venue'])
Obtendo jogadores do America Mineiro...
dict_keys(['team', 'venue'])
Obtendo jogadores do Sao Paulo...
dict_keys(['team', 'venue'])
Obtendo jogadores do Flamengo...
dict_keys(['team', 'venue'])
Obtendo jogadores do Santos...
dict_keys(['team', 'venue'])
Obtendo jogadores do Gremio...
dict_keys(['team', 'venue'])
Obtendo jogadores do Corinthians...
dict_keys(['team', 'venue'])
Obtendo jogadores do Vasco DA Gama...
dict_keys(['team', 'venue'])
Obtendo jogadores do Atletico Paranaense...
dict_keys(['team', 'venue'])
Obtendo jogadores do Cruzeiro...
dict_keys(['team', 'venue'])
Obtendo jogadores do Coriti

In [ ]:
for team, info in data["teams"].items():
    print(f'{team} - {info["players"]}')

In [25]:
client.debug_print_endpoint_keys(sample_size=3)


ENDPOINT: leagues
Itens retornados: 1 (mostrando até 3 para inspeção)
country
country.code
country.flag
country.name
league
league.id
league.logo
league.name
league.type
seasons
seasons[]
seasons[].coverage
seasons[].coverage.fixtures
seasons[].coverage.fixtures.events
seasons[].coverage.fixtures.lineups
seasons[].coverage.fixtures.statistics_fixtures
seasons[].coverage.fixtures.statistics_players
seasons[].coverage.injuries
seasons[].coverage.odds
seasons[].coverage.players
seasons[].coverage.predictions
seasons[].coverage.standings
seasons[].coverage.top_assists
seasons[].coverage.top_cards
seasons[].coverage.top_scorers
seasons[].current
seasons[].end
seasons[].start
seasons[].year

ENDPOINT: teams
Itens retornados: 20 (mostrando até 3 para inspeção)
team
team.code
team.country
team.founded
team.id
team.logo
team.name
team.national
venue
venue.address
venue.capacity
venue.city
venue.id
venue.image
venue.name
venue.surface

ENDPOINT: players

----------------------------------------